## Preprocess the Sim2Real dataset

The Sim2Real dataset uses jpeg images to represent wildfire masks. This representation is very lightweight but slow to work with. 
To benchmark faster, we convert them to binary NumPy files. These files are ~100 times heavier and it requires an overhead time to convert from jpeg to them, but once they are created and stored, they allow 50--100x faster operations.

1. Download the dataset from [Sim2Real-Fire GitHub repository](https://github.com/TJU-IDVLab/Sim2Real-Fire).
2. Extract all files from the dataset into your `Dataset` folder
3. Run the preprocessing function to convert from the jpeg representation to the NumPy one, and compute burn maps

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import required modules
import sys
import os
import time

# Add code to path
module_path = os.path.abspath(".") + "/code"
if module_path not in sys.path:
    sys.path.append(module_path)

from dataset import preprocess_sim2real_dataset, load_scenario_npy
from Strategy import SensorPlacementStrategy, DroneRoutingStrategy, return_no_custom_parameters
from benchmark import run_benchmark_scenario,run_benchmark_scenarii_sequential #benchmark_on_sim2real_dataset
from displays import create_scenario_video

In [5]:
# n_max_scenarii_per_layout controls the number of scenarios we convert from jpeg to NumPy files for each layout.
# REQUIRES 400 GB of STOCKAGE and ~15 mins
preprocess_sim2real_dataset("Dataset/", n_max_scenarii_per_layout=100) 

Converting JPG scenarios to NPY for Dataset/


100it [00:40,  2.49it/s]
100it [00:29,  3.39it/s]
100it [00:26,  3.74it/s]


Computing burn maps...
Computing burn map for Dataset/0012_02094/scenarii/ and files with extension .npy


100%|██████████| 34/34 [00:00<00:00, 35.30it/s]


Computing burn map for Dataset/0004_01191/scenarii/ and files with extension .npy


...

100%|██████████| 100/100 [00:01<00:00, 66.34it/s]


Computing burn map for Dataset/0005_00725/scenarii/ and files with extension .npy


100%|██████████| 100/100 [00:01<00:00, 81.69it/s]


Computing burn map for Dataset/0001/scenarii/ and files with extension .npy


100%|██████████| 100/100 [00:00<00:00, 222.54it/s]


We can now run the benchmark function on any scenario.
1. We load the scenario using `load_scenario_npy` since it is in preprocessed `npy` format
2. Werun the benchmark using `run_benchmark_scenario` that takes as input:
    - The `scenario`, 
    - The sensor placement strategy and the drone routing strategy
    - A dictionary `custom_initialization_parameters` that contains any custom initialization inputs for your strategy functions (such as the burn map if your strategy needs a burn map as input)
    - A python function `custom_step_parameters_function` that returns a dictionary of custom inputs for your routing function. This function is executed by the benchmarking code at each time step. Your routing function will be called with: `routing(custom_step_parameters_function())` internally

In [6]:
# That's very fast to run!
time_start = time.time()
scenario = load_scenario_npy("Dataset/0002/scenarii/0002_00010.npy")
device, delta_t, _ = run_benchmark_scenario(scenario, SensorPlacementStrategy, DroneRoutingStrategy, custom_initialization_parameters = None, custom_step_parameters_function = return_no_custom_parameters)
print("Fire detected in ", delta_t, "time steps by device: ", device)
print(f"Time taken to run benchmark on the scenario: {time.time() - time_start} seconds")


Fire detected in  drone time steps by device:  15
Time taken to run benchmark on the scenario: 0.008838176727294922 seconds


We can visualize the strategy in action by creating a video of the scenario. 
1. Use the `return_history` parameter or `run_benchmark_scenario` to output the log of ground sensor and drone positions during the benchmark
2. Use the `create_scenario_video` function to compile the video. This can take a couple seconds

In [7]:
delta , device , (position_history, ground, charging)  = run_benchmark_scenario(scenario, SensorPlacementStrategy, DroneRoutingStrategy, custom_initialization_parameters = None, custom_step_parameters_function = return_no_custom_parameters, return_history=True)
create_scenario_video(scenario[:len(position_history)],drone_locations_history=position_history,starting_time=0,out_filename='test_simulation', ground_sensor_locations = ground, charging_stations_locations = charging)

Video saved at: display_test_simulation/test_simulation.mp4


Instead of running a benchmark on a single scenario, we are interested in running the benchmark on all scenarii of a given layout (potentially in parallel!)

1. We use `run_benchmark_scenarii_sequential`

In [13]:
class SensorPlacementStrategy(): # ----- D: change to SensorPlacementStrategyRandom
    def __init__(self, automatic_initialization_parameters:dict, custom_initialization_parameters:dict):
        """
        Initialize the ground placement strategy using random placement.
        
        Args:
            automatic_initialization_parameters: dict with keys:
                "N": Grid height
                "M": Grid width
                "max_battery_distance": int
                "max_battery_time": int
                "n_drones": int
                "n_ground_stations": Target number of ground stations
                "n_charging_stations": Target number of charging stations
                "ground_sensor_locations": list of tuples (x,y)
            custom_initialization_parameters: dict
        Returns:
            ground_sensor_locations: list of tuples (x,y)
            charging_station_locations: list of tuples (x,y)
        """
        # Generate random positions using list comprehensions
        # YOUR CODE HERE
        self.ground_sensor_locations = [(random.randint(0, automatic_initialization_parameters["N"]-1), 
                                       random.randint(0, automatic_initialization_parameters["M"]-1)) 
                                      for _ in range(automatic_initialization_parameters["n_ground_stations"])]
        
        self.charging_station_locations = [(random.randint(0, automatic_initialization_parameters["N"]-1), 
                                          random.randint(0, automatic_initialization_parameters["M"]-1)) 
                                         for _ in range(automatic_initialization_parameters["n_charging_stations"])]

    def get_locations(self):
        """
        Returns the locations of the ground sensors and charging stations
        """
        # Do not overwrite this function
        return self.ground_sensor_locations, self.charging_station_locations

class SensorPlacementOptimization(SensorPlacementStrategy):
    def __init__(self, automatic_initialization_parameters:dict, custom_initialization_parameters:dict):
        """
        Initialize the ground placement strategy using Julia's optimization model.
        
        Args:
            automatic_initialization_parameters: dict with keys:
                "n_ground_stations": Target number of ground stations
                "n_charging_stations": Target number of charging stations
                "N": Grid height
                "M": Grid width
            custom_initialization_parameters: dict with keys:
                "risk_pertime_dir": Directory containing burn map files
        """
        # Initialize empty lists (skip parent's random initialization)
        self.ground_sensor_locations = []
        self.charging_station_locations = []

        if "risk_pertime_dir" not in custom_initialization_parameters:
            raise ValueError("risk_pertime_dir is not defined")

        if not custom_initialization_parameters["risk_pertime_dir"].endswith("/"):
            custom_initialization_parameters["risk_pertime_dir"] += "/"
        # Load the Julia module and function
        # jl.include("julia/ground_charging_opt.jl") # Done in julia_caller
        
        # Call the Julia optimization function
        x_vars, y_vars = jl.ground_charging_opt_model_grid(custom_initialization_parameters["risk_pertime_dir"], automatic_initialization_parameters["n_ground_stations"], automatic_initialization_parameters["n_charging_stations"]) # ----- D: why is N,M gridsize not an input?
        # save the result in a json file
        with open(custom_initialization_parameters["risk_pertime_dir"][:-1] + "_ground_sensor_locations.json", "w") as f:
            json.dump(x_vars, f)
        with open(custom_initialization_parameters["risk_pertime_dir"][:-1] + "_charging_station_locations.json", "w") as f:
            json.dump(y_vars, f)
        
        self.ground_sensor_locations = list(x_vars)
        self.charging_station_locations = list(y_vars)


In [14]:
run_benchmark_scenarii_sequential("Dataset/0002/scenarii/", SensorPlacementOptimization, DroneRoutingStrategy, custom_initialization_parameters_function = lambda x: None, custom_step_parameters_function = return_no_custom_parameters, starting_time=0, max_n_scenarii=100)

  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: argument of type 'NoneType' is not iterable

On all of the dataset!

In [10]:
def benchmark_on_sim2real_dataset(dataset_folder_name, ground_placement_strategy, drone_routing_strategy, custom_initialization_parameters_function, custom_step_parameters_function, max_n_scenarii=None, starting_time=0):
    """
    Run benchmarks on a simulation-to-real-world dataset structure.

    Args:
        dataset_folder_name (str): Root folder containing layout folders with scenario data.
        ground_placement_strategy (function): Strategy for placing ground sensors and charging stations.
        drone_routing_strategy (function): Strategy for controlling drone movements.
        ground_parameters (tuple): Parameters for ground placement strategy.
        routing_parameters (tuple): Parameters for routing strategy.
        max_n_scenarii (int, optional): Maximum number of scenarios to process per layout. If None, processes all scenarios.
        starting_time (int, optional): Time step at which the wildfire starts.
    """
    if not dataset_folder_name.endswith('/'):
        dataset_folder_name += '/'
    
    for layout_folder in os.listdir(dataset_folder_name):
        if not os.path.exists(dataset_folder_name + layout_folder + "/scenarii/"):continue
        run_benchmark_scenarii_sequential(dataset_folder_name + layout_folder + "/scenarii/",
                                          ground_placement_strategy, 
                                          drone_routing_strategy, 
                                          custom_initialization_parameters_function, 
                                          custom_step_parameters_function, 
                                          starting_time=starting_time, 
                                          max_n_scenarii=max_n_scenarii)

In [11]:
benchmark_on_sim2real_dataset("Dataset/", SensorPlacementStrategy, DroneRoutingStrategy, custom_initialization_parameters_function = lambda x: None, custom_step_parameters_function = return_no_custom_parameters, max_n_scenarii=100, starting_time=0)

 34%|███▍      | 34/100 [00:00<00:01, 63.98it/s]


This strategy took on average 2.6363636363636362 time steps to find the fire.
Fire found 13.0% of the time by ground sensor
Fire found 12.0% of the time by charging station
Fire found 8.0% of the time by drone
Fire found 1.0% of the time by undetected


 52%|█████▏    | 52/100 [00:00<00:00, 54.51it/s]


This strategy took on average 3.7604166666666665 time steps to find the fire.
Fire found 22.0% of the time by ground sensor
Fire found 14.0% of the time by charging station
Fire found 12.0% of the time by drone
Fire found 4.0% of the time by undetected


100%|██████████| 100/100 [00:00<00:00, 216.27it/s]


...

This strategy took on average 26.703296703296704 time steps to find the fire.
Fire found 28.0% of the time by ground sensor
Fire found 19.0% of the time by charging station
Fire found 44.0% of the time by drone
Fire found 9.0% of the time by undetected


100%|██████████| 100/100 [00:01<00:00, 87.65it/s]


This strategy took on average 19.96511627906977 time steps to find the fire.
Fire found 34.0% of the time by ground sensor
Fire found 16.0% of the time by charging station
Fire found 36.0% of the time by drone
Fire found 14.0% of the time by undetected


100%|██████████| 100/100 [00:00<00:00, 252.55it/s]

This strategy took on average 12.72 time steps to find the fire.
Fire found 31.0% of the time by ground sensor
Fire found 23.0% of the time by charging station
Fire found 46.0% of the time by drone
Fire found 0.0% of the time by undetected
